In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
top = pd.read_csv('../input/restaurant-business-rankings-2020/Top250.csv')
indep =  pd.read_csv('../input/restaurant-business-rankings-2020/Independence100.csv')

In [ ]:
top.head(10)

In [ ]:
top.info()

In [ ]:
top[top.isnull()==False].count()

Content, Headquaters have Null values

# 1. To see 'Segment_Category' distribution

In [ ]:
category_top = top.groupby(by=['Segment_Category']).Sales.sum().sort_values(ascending=False).head(10).reset_index()

In [ ]:
import plotly.express as px
px.bar(category_top, x='Sales', y='Segment_Category', title='Segment Category')

# It seems like 'Quick Service & Burger' beats the others overwhelminglly, so let's check category into services and foods'kind

In [ ]:
service = []
for i in range(len(top)):
    if len(top.Segment_Category.iloc[i].split('&')) != 1:
        service.append(top.Segment_Category.iloc[i].split('&')[0])
    else:
        service.append('NaN')
top['Services'] = service        
top['Food_kinds'] =  top.Segment_Category.apply(lambda d:d.split('&')[-1]).apply(lambda d:d.strip())

In [ ]:
top_Services = top.groupby(by='Services').Sales.sum().sort_values(ascending=False).reset_index()
top_Foods = top.groupby(by='Food_kinds').Sales.sum().sort_values(ascending=False).reset_index()
top_Foods.Food_kinds.apply(lambda d:d.rstrip())
px.bar(top_Services, x='Sales', y='Services', title='Services and Salses Relation', width=750, height=500)

# Now that **Quick Service** is placed absolutely top, 
# this is because of COVID-19.

# We've come to order takeaway more than past.

In [ ]:
px.bar(top_Foods, x='Sales', y='Food_kinds', title='Kind of Foods and Salses Relation', width=750, height=500)

In [ ]:
px.pie(top_Foods, values='Sales', names='Food_kinds',title='Kind of Foods and Salses Relation with pie graph',width=750, height=500)

# Burger has the biggest sales. 🍔

# But if you want to set up your restaurant or so, you need to take into consideration of what is going to be popular as well.

# This thought can help you to differenciate your bussiness from already popular restaurant.

# *So just consider => How about 'increasing foods'kind?*

# 🍕，🐟、🥬．．．．．．．．？

In [ ]:
top['YOY_Sales'] = top['YOY_Sales'].apply(lambda d: float(d.split('%')[0]))

In [ ]:
top_yoy_sales = top.groupby(by = 'Food_kinds').YOY_Sales.mean().sort_values(ascending=False).reset_index()

In [ ]:
px.bar(top_yoy_sales, x='Food_kinds', y='YOY_Sales', color='YOY_Sales')

# Healthy food has gone viral🥬
# This yields simple guess that those who are aspiring to spend healthy life have been increasing

Burger sales:84889, Healthy:184 in this year, however the growth of former is 3.27 and the later is 16.8

In [ ]:
import plotly.graph_objects as go

burger = [84889]
healthy = [184]
chicken = [29153]
for i in range(20):
    burger.append(burger[-1]*(1.0327))
    healthy.append(healthy[-1]*(1.168))
    chicken.append(chicken[-1]*(1.74))
fig = go.Figure()
# Create and style traces
fig.add_trace(go.Scatter(x=[i for i in range(20)], y=burger, name='Burger',
                         line=dict(color='firebrick', width=4)))
fig.add_trace(go.Scatter(x=[i for i in range(20)], y=healthy, name = 'Healthy',
                         line=dict(color='royalblue', width=4)))
fig.add_trace(go.Scatter(x=[i for i in range(20)], y=chicken, name = 'Chicken',
                         line=dict(color='rgb(67,67,67)', width=4)))

fig.update_layout(title='Burger, Chicken and Healthy feature prediction(simple)',
                   xaxis_title='year',
                   yaxis_title='Salses')


fig.show()

This is quite rough prediction.
Burger has down-to-earth sales in each years, so It would not be predicted to plunge.
However Healthy would not be a prospect to get big sales in near feature unless it goes vital beacuse the initial value is so small. It needs to gain a momuntum to make a healty-businesses bigger.

Chicken(Quick service would be better) has a relatively high increasing rate moreover had high Sales already, thus It could be the best kind of food to choose restaurant type.

# But what is important thing is that currently Healthy food has got a momentum to be a mainstream. So in general ,Burgers and Pizzas are thought to be 'UNHEALTHY', so both sales might not be able to increase as much as cureent sales. 
# 🍔，🍕　＜＝　　🥬

# 2. # Positive unit rate and YOY_sales is going to expand thier business more and more

In [ ]:
top.YOY_Units = top.YOY_Units.apply(lambda d:float(d.split('%')[0]))

In [ ]:
top_sales_unit = top[(top.YOY_Sales > 0) & (top.YOY_Units > 0)]

In [ ]:
top_sales_unit['YOY'] = top_sales_unit.YOY_Sales + top_sales_unit.YOY_Units

In [ ]:
top_YOY = top_sales_unit.sort_values(by='YOY', ascending=False)
top_YOY.head(10)

# What Restaurant is a good prospect is Chicken Salad Chick, since this is the type of a chicken shop and has a word salad which might be able to induce those who want to take in a healty thing!!
# 🍗 and 🥗 combination is the best

> # 3.  Feature analysis

In [ ]:
feature = pd.read_csv('../input/restaurant-business-rankings-2020/Future50.csv')
feature.head(10)

#  Analysis of Sales and Unis correlation

In [ ]:
feature['States'] = feature.Location.apply(lambda d: d.split(',')[-1].lstrip().replace('.', ''))

In [ ]:
feature['YOY_Sales'] = feature['YOY_Sales'].apply(lambda d:float(d.split('%')[0]))
feature['YOY_Units'] = feature['YOY_Units'].apply(lambda d:float(d.split('%')[0]))

In [ ]:
feature['YOY_Sales'] = feature['YOY_Sales'].apply(lambda d:d*0.01)
feature['YOY_Units'] = feature['YOY_Units'].apply(lambda d:d*0.01)

In [ ]:
feature['Previous_Sales'] = (feature.Sales/(feature.YOY_Sales+1)).round(2)
feature['Previous_Units'] = (feature.Units/(feature.YOY_Units+1)).round(2)
feature.head(10)

In [ ]:
this_two_years_growth = feature.groupby(by=['States'])[['Sales', 'Previous_Sales']].sum().reset_index()

In [ ]:
this_two_years_growth['diff'] = this_two_years_growth['Sales'] - this_two_years_growth['Previous_Sales']
this_two_years_growth.sort_values(by=['diff'], ascending=False, inplace=True)
this_two_years_growth = this_two_years_growth.reset_index()
this_two_years_growth.drop(columns=['index'], inplace=True)
px.scatter(this_two_years_growth, x="Previous_Sales", y="Sales", color="diff")

# This is feature premise restaurant data, so the correlation between previous and current year can be nearly 1.

In [ ]:
import seaborn as sns
this_two_years_growth_corr = this_two_years_growth.iloc[:,1:]

In [ ]:
corr = this_two_years_growth_corr.corr()
sns.heatmap(corr, square=True, annot=True)

How about Unit

In [ ]:
unit_two_years_growth = feature.groupby(by=['States'])[['Units', 'Previous_Units']].sum().reset_index()
unit_two_years_growth['unit_diff'] = unit_two_years_growth['Units'] - unit_two_years_growth['Previous_Units']
unit_two_years_growth.sort_values(by=['unit_diff'], ascending=False, inplace=True)
unit_two_years_growth = unit_two_years_growth.reset_index()
unit_two_years_growth.drop(columns=['index'], inplace=True)
px.scatter(unit_two_years_growth, x="Previous_Units", y="Units", color="unit_diff", title='2 years Unit relation')

This has also the same relation as sales which means this correlation is nealy 1

In [ ]:
px.pie(this_two_years_growth.head(10), values='Sales', names='States', title='Current year sales in each states')

In [ ]:
px.pie(this_two_years_growth.head(10), values='Previous_Sales', names='States', title='Previous year sales in each states')

In [ ]:
px.pie(this_two_years_growth.head(10), values='diff', names='States', title='Difference year sales in each states')

# The down_to_earth place to set your business are apparently NY or California. But where has a big growth rate is NC, so the feature prediction that NC is potential to surpass both two states.

# ** Conclusion

# 1. Healty food will be going mainstream
# 2. Quick Service Sales is goin to be more occupying the whole service
# 3. 1+2 => Healthy, Quick and Chicken restaurant can get good popularity.
# 4. It is good to focus on Takeaway service since we are in the COVID situation
# => But when it comes to placing your restaurant, NY or California would be nice.
# => Given feature potential city, NC is also good selection👍